In [1]:
import pandas as pd 
import numpy as np
import re
import csv
from sklearn.model_selection import train_test_split
from collections import Counter

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 200)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
path = '/mnt/files/workdata/work/python-scripts/prediction_analyzer/predict_stock_quotes/data/data-2021-06-10/'
file_new = path + 'trainingset _1_.xlsx'
df_new = pd.read_excel(file_new, dtype=str)
print(df_new.shape)
#29006

(29006, 6)


In [100]:
path = '/mnt/files/workdata/work/python-scripts/prediction_analyzer/predict_stock_quotes/data/'

file_old = path + '21K-predict.csv'
df_old = pd.read_csv(file_old,  dtype=str)

print(df_old.shape)
# (21129, 8)

(21129, 8)


In [101]:
df_old_sub = df_old[['title', 'Unnamed: 2']].copy()
df_old_sub.columns = ['title', 'SENTIMENT']

In [4]:
df_new_sub = df_new[['title', 'znak']].copy()
df_new_sub.columns = ['title', 'SENTIMENT']

#### For new data

In [6]:
# mask = df_new_sub['title'].isin(df_old_sub['title'])
# df_unique =  df_new_sub[~ mask].copy()
# print(df_unique.shape[0])

#### For old data

In [7]:
df_unique = df_new_sub
print(df_unique.shape[0])

29006


In [8]:
df_benzinga = df_unique.copy()
df_benzinga.head()

,title,SENTIMENT
0,ChipMOS misses on revenue,2
1,Reuters Reported Foxconn\'s iPhone Production at India Factory Falls More Than 50% Due to Surge in Covid-19 Infections Among Workers,2
2,"Allot EPS beats by $0.06, misses on revenue; provides outlook",2
3,Reuters Reported Tesla Put Plans to Buy Land to Expand Shanghai Plant on Hold Due to U.S.-China Tensions -; Tesla Considered Exporting Model 3 Vehicles from China to the U.S.,1
4,Nasdaq futures continue selloff as tech drags broader market lower,2


In [9]:
mask = (df_benzinga.SENTIMENT == '0') | (df_benzinga.SENTIMENT == '1') | (df_benzinga.SENTIMENT == '2')
df_benzinga = df_benzinga[mask].copy()

In [10]:
print(df_benzinga.shape[0])
print(Counter(df_benzinga.SENTIMENT))

28827
Counter({'1': 12663, '0': 12252, '2': 3912})


In [11]:
df_benzinga.drop_duplicates(subset='title', inplace=True)

In [12]:
df_benzinga.shape[0]

28325

In [13]:
df_benzinga['sentiment_words'] = np.where(df_benzinga['SENTIMENT'] == '0', 'neutral', np.where(df_benzinga['SENTIMENT'] == '1', 'positive', 'negative'))

In [14]:
Counter(df_benzinga.sentiment_words)

Counter({'negative': 3890, 'positive': 12611, 'neutral': 11824})

In [15]:
df_benzinga = df_benzinga[['title', 'sentiment_words']].copy()

In [112]:
separator = '@'

In [113]:
sep_find = df_benzinga.title.map(lambda x: ''.join(re.findall(separator, str(x))))

In [114]:
sep_find.value_counts()

        28024
@         292
@@          4
@@@         3
@@@@        2
Name: title, dtype: int64

In [115]:
df_benzinga.title = df_benzinga.title.map(lambda x: re.sub(separator, '',str(x)))

In [116]:
find_symbols = df_benzinga[df_benzinga.title.str.contains(r'\\n')]

In [117]:
find_symbols = find_symbols.title.values[0]
find_symbols

'Vodafone Reports Its Egypt Unit To Buy Spectrum, Payments Will Be Phased Over 3 Years, With Initial Payment Of\\n $270M'

In [118]:
BACKSLASH_n = r'\\n'

In [119]:
re.sub(BACKSLASH_n, '', find_symbols)

'Vodafone Reports Its Egypt Unit To Buy Spectrum, Payments Will Be Phased Over 3 Years, With Initial Payment Of $270M'

In [120]:
df_benzinga.title = df_benzinga.title.map(lambda x: re.sub(BACKSLASH_n, '',str(x)))

In [121]:
df_benzinga.title = df_benzinga.title.map(lambda x: str(x).strip('\n'))

In [122]:
find_symbols = df_benzinga[df_benzinga.title.str.contains('JPMorgan')]

In [123]:
find_symbols = find_symbols.title.values[0]
find_symbols

'Cree stock gains after losing JPMorgan bear on improved valuation'

In [124]:
DOUBLE_QUOTES = r'"'

In [125]:
re.sub(DOUBLE_QUOTES, '', find_symbols)

'Cree stock gains after losing JPMorgan bear on improved valuation'

In [126]:
df_benzinga.title = df_benzinga.title.map(lambda x: re.sub(DOUBLE_QUOTES, '', str(x)))

In [127]:
df_benzinga.reset_index(drop=True, inplace=True)

In [128]:
df_benzinga.head()

,title,sentiment_words
0,ChipMOS misses on revenue,negative
1,Reuters Reported Foxconn\'s iPhone Production at India Factory Falls More Than 50% Due to Surge in Covid-19 Infections Among Workers,negative
2,"Allot EPS beats by $0.06, misses on revenue; provides outlook",negative
3,Reuters Reported Tesla Put Plans to Buy Land to Expand Shanghai Plant on Hold Due to U.S.-China Tensions -; Tesla Considered Exporting Model 3 Vehicles from China to the U.S.,positive
4,Nasdaq futures continue selloff as tech drags broader market lower,negative


### Save as train or test

In [71]:
df_benzinga.columns = ['text', 'label']

In [73]:
df_benzinga.to_csv('data/sentiment_data/train.csv', sep='\t', header=True, index=True, quoting = csv.QUOTE_NONE, escapechar = ' ')

In [ ]:
stop

### Save for split teach/test

In [129]:
df_benzinga['text_label'] = df_benzinga.title +' '+ separator + df_benzinga.sentiment_words

In [130]:
df_benzinga.head(3)

,title,sentiment_words,text_label
0,ChipMOS misses on revenue,negative,ChipMOS misses on revenue @negative
1,Reuters Reported Foxconn\'s iPhone Production at India Factory Falls More Than 50% Due to Surge in Covid-19 Infections Among Workers,negative,Reuters Reported Foxconn\'s iPhone Production at India Factory Falls More Than 50% Due to Surge in Covid-19 Infections Among Workers @negative
2,"Allot EPS beats by $0.06, misses on revenue; provides outlook",negative,"Allot EPS beats by $0.06, misses on revenue; provides outlook @negative"


In [131]:
benzinga_text = df_benzinga['text_label']

In [133]:
file_out = 'data/text_data_for_split/benzinga_text_labels.txt'
                     
with open(file_out, 'w') as filehandle:
    for listitem in  df_benzinga['text_label'].values:
        filehandle.write('%s\n' % listitem)

### Experiments with prepare data.

In [17]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import collections

[nltk_data] Downloading package stopwords to /home/sergey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
# Continue from str with separator
df = df_benzinga.copy()
df.columns = ['text', 'sentiment_words']
df.text = df.text.astype(str)
df.reset_index(drop=True, inplace=True)
df.head()

,text,sentiment_words
0,ChipMOS misses on revenue,negative
1,Reuters Reported Foxconn\'s iPhone Production at India Factory Falls More Than 50% Due to Surge in Covid-19 Infections Among Workers,negative
2,"Allot EPS beats by $0.06, misses on revenue; provides outlook",negative
3,Reuters Reported Tesla Put Plans to Buy Land to Expand Shanghai Plant on Hold Due to U.S.-China Tensions -; Tesla Considered Exporting Model 3 Vehicles from China to the U.S.,positive
4,Nasdaq futures continue selloff as tech drags broader market lower,negative


In [37]:
ticker_pattern = re.compile(r'(^\$[A-Z]+|^\$ES_F)')
ht_pattern = re.compile(r'#\w+')

ticker_dic = collections.defaultdict(int)
ht_dic = collections.defaultdict(int)

for text in df['text']:
    for word in str(text).split():
        if ticker_pattern.fullmatch(word) is not None:
            ticker_dic[word[1:]] += 1
        
        word = word.lower()
        if ht_pattern.fullmatch(word) is not None:
            ht_dic[word] += 1

In [38]:
char_digit_percent_only = re.compile(r'[^a-zA-Z\s0-9%]')

In [39]:
handle_pattern = re.compile(r'@\w+')
emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
url_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
pic_pattern = re.compile('pic\.twitter\.com/.{10}')
special_code = re.compile(r'(&amp;|&gt;|&lt;)')
tag_pattern = re.compile(r'<.*?>')

STOPWORDS = set(stopwords.words('english')).union(
    {'rt', 'retweet', 'RT', 'Retweet', 'RETWEET'})

lemmatizer = WordNetLemmatizer()

In [40]:
def hashtag(phrase):
    return ht_pattern.sub(' ', phrase)
def remove_ticker(phrase):
    return ticker_pattern.sub('', phrase)
def specialcode(phrase):
    return special_code.sub(' ', phrase)
def emoji(phrase):
    return emoji_pattern.sub(' ', phrase)
def url(phrase):
    return url_pattern.sub('', phrase)
def pic(phrase):
    return pic_pattern.sub('', phrase)
def html_tag(phrase):
    return tag_pattern.sub(' ', phrase)
def handle(phrase):
    return handle_pattern.sub('', phrase)

In [41]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub('DIS', 'Disney', phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"(he|He)\'s", "he is", phrase)
    phrase = re.sub(r"(she|She)\'s", "she is", phrase)
    phrase = re.sub(r"(it|It)\'s", "it is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"(\'ve|has)", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def onlychardigit(phrase):
    return char_digit_percent_only.sub('', phrase)

def remove_stopwords(phrase):
    return " ".join([word for word in str(phrase).split()\
                     if word not in STOPWORDS])

def tokenize_stem(phrase):   
    tokens = word_tokenize(phrase)
    stem_words =[]
    for token in tokens:
        word = lemmatizer.lemmatize(token)
        stem_words.append(word)        
    buf = ' '.join(stem_words)    
    return buf

In [42]:
def arrange_text(df):
    df['text2'] = df['text'].apply(emoji)
    df['text2'] = df['text2'].apply(handle)
    df['text2'] = df['text2'].apply(specialcode)
    df['text2'] = df['text2'].apply(hashtag)
    df['text2'] = df['text2'].apply(url)
    df['text2'] = df['text2'].apply(pic)
    df['text2'] = df['text2'].apply(html_tag)
    df['text2'] = df['text2'].apply(onlychardigit)
    df['text2'] = df['text2'].apply(decontracted)
    df['text2'] = df['text2'].apply(onlychardigit)
    df['text2'] = df['text2'].apply(tokenize_stem)
    df['text2'] = df['text2'].apply(remove_stopwords)

In [47]:
arrange_text(df)

In [48]:
df.text = df.text2
df.drop(columns=['text2'], inplace=True)
df.rename(columns={'sentiment_words':'label'}, inplace=True)

In [50]:
df.head()

,text,label
0,ChipMOS miss revenue,negative
1,Reuters Reported Foxconns iPhone Production India Factory Falls More Than 50 % Due Surge Covid19 Infections Among Workers,negative
2,Allot EPS beat 006 miss revenue provides outlook,negative
3,Reuters Reported Tesla Put Plans Buy Land Expand Shanghai Plant Hold Due USChina Tensions Tesla Considered Exporting Model 3 Vehicles China US,positive
4,Nasdaq future continue selloff tech drag broader market lower,negative


In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(df.text, df.label, test_size=0.3, random_state=123)

In [57]:
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5, random_state=123)

In [65]:
df_train = pd.merge(X_train, Y_train, how='left', left_index=True, right_index=True)
df_test = pd.merge(X_test, Y_test, how='left', left_index=True, right_index=True)
df_validation = pd.merge(X_val, Y_val, how='left', left_index=True, right_index=True)

In [67]:
df_train.head(1)

,text,label
19368,Following pressure Biogen start compassionate use program ALS drug tofersen,neutral


In [68]:
# df_train.to_csv('data/csv_files/train_28K_clear_arrange.csv', sep='\t', header=True, index=True, quoting = csv.QUOTE_NONE, escapechar = ' ')
# df_test.to_csv('data/csv_files/test_28K_clear_arrange.csv', sep='\t', header=True, index=True, quoting = csv.QUOTE_NONE, escapechar = ' ')
# df_validation.to_csv('data/csv_files/validation_28K_clear_arrange.csv', sep='\t', header=True, index=True, quoting = csv.QUOTE_NONE, escapechar = ' ')